In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from nltk import FreqDist
from nltk.corpus import stopwords
import os 
import re
from operator import itemgetter
import datetime
from dateutil.parser import parse
import math
import pickle
from gensim.models import KeyedVectors
import sys
np.set_printoptions(threshold=sys.maxsize)
import time

In [8]:
data =  pd.read_pickle('./data_model_fc.pkl')

In [2]:
en_model = KeyedVectors.load_word2vec_format('cc.de.300.vec')

/Users/adrian/coding/rtbda_seminar/rtbda_venv/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
en_model.most_similar(positive=['hund'], topn=5)
#most_similar(positive=[WORD], topn=N)

[('hunde', 0.7181448340415955),
 ('hunden', 0.7109783291816711),
 ('Hund', 0.696244478225708),
 ('hund.', 0.6680127382278442),
 ('welpe', 0.6609440445899963)]

In [11]:
en_model.most_similar('haus')

[('Haus', 0.6784808039665222),
 ('haus.', 0.6606301665306091),
 ('häuser', 0.6593822240829468),
 ('holzhaus', 0.6582863330841064),
 ('grundstück', 0.6517457962036133),
 ('hauses', 0.6508925557136536),
 ('häuschen', 0.6424012184143066),
 ('zimmer', 0.6324517726898193),
 ('wohnung', 0.6287425756454468),
 ('gartenhaus', 0.6262539029121399)]

In [10]:
#preprocess words
word_emb_input = pd.DataFrame(data.loc[:,'word'].str.lower())
word_emb_input = pd.DataFrame(word_emb_input)
word_emb_input = word_emb_input.astype(str)
word_emb_input['preprocessed'] = np.nan
word_emb_input ['preprocessed'] = [re.sub('\d', 'D', x) for x in word_emb_input['word'].tolist()]
word_emb_input = word_emb_input.drop(['word'],1)

In [14]:
missing=[0]*300
def fun(key):
    try:
        return(en_model[key])
    except:
        return(missing)

In [15]:
word_emb_input['vector'] = word_emb_input['preprocessed'].apply(fun)
df = pd.DataFrame(word_emb_input['vector'].values.tolist())
df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.0084,0.0145,-0.0036,-0.0082,-0.0220,-0.0251,0.0001,-0.0135,0.0063,0.0220,...,0.0031,0.0079,-0.0037,0.0001,0.0197,-0.0091,-0.0019,-0.0028,-0.0201,-0.0021
1,-0.0444,-0.0323,-0.0859,-0.0301,-0.0662,0.0563,0.0431,-0.0037,0.0140,0.0413,...,0.0126,0.0312,0.0265,-0.0110,-0.0008,-0.0370,-0.0298,0.0820,0.0704,-0.0192
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,-0.0940,0.0238,0.0430,-0.0127,-0.0133,0.0654,-0.0787,0.0691,-0.0593,0.1505,...,0.0439,0.0491,0.0298,0.0671,0.1457,0.0384,-0.0383,0.0704,-0.0972,-0.0809
5,-0.1406,-0.3137,-0.2052,-0.2788,-0.2393,0.1805,0.0400,0.1415,-0.0547,0.0647,...,0.1374,-0.0332,0.0733,0.0122,-0.2553,-0.1791,-0.1202,-0.1009,-0.3689,0.0950
6,-0.0747,0.0011,0.0081,0.0371,0.0257,0.2607,0.0084,-0.0758,0.0222,-0.1083,...,0.0155,0.0526,0.0127,-0.0590,-0.1364,-0.0619,-0.0338,0.0517,0.0655,-0.0553
7,-0.1406,-0.3137,-0.2052,-0.2788,-0.2393,0.1805,0.0400,0.1415,-0.0547,0.0647,...,0.1374,-0.0332,0.0733,0.0122,-0.2553,-0.1791,-0.1202,-0.1009,-0.3689,0.0950
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,-0.0392,-0.1829,-0.2038,-0.0265,0.0237,0.1201,0.0804,-0.0434,0.0759,-0.1896,...,-0.0743,-0.0307,-0.0281,0.0546,-0.1684,-0.0313,0.0268,-0.1611,-0.1354,0.0988


In [ ]:
#To slow
'''
words = list(data.loc[:,'word'])
emb = np.array(np.full((1, 300), 0.0))
nv = np.array(np.full((1, 300), 0.0))

i = 0
for x in words:
    print (i)
    try:
        emb = np.append (emb, np.array([en_model[x]]), axis = 0)
    except:
        emb = np.append (emb, nv, axis = 0)
    i +=1
'''